In [1]:
from langchain_ollama import ChatOllama
import re
import httpx

In [15]:
client = ChatOllama(model = "qwen2.5",
                   temperature = 0)
chat_completion = client.invoke(
    input = [{"role": "user", "content": "Hellow world"}]
)

In [16]:
chat_completion.content

'Hello! How can I assist you today? If you meant to say "Hello, world!" in the context of programming, it\'s a common first program used to demonstrate the syntax of a programming language. Would you like an example in a specific language?'

# Make custom agent class

In [17]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    def execute(self):
        completion = client.invoke(
            input = self.messages
        )
        return completion.content

In [70]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Strictly follow the above instructions
Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [32]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [33]:
abot = Agent(prompt)

In [34]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [35]:
result = average_dog_weight("Toy Poodle")
print(result)

a toy poodles average weight is 7 lbs


In [36]:
next_prompt = "Observation: {}".format(result)

In [38]:
abot(next_prompt)

'Answer: A toy poodle weighs approximately 7 lbs.'

In [39]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\nStrictly follow the above instructions\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role'

# Example 2

In [40]:
#We need to reinitialize the agent
abot = Agent(prompt)

In [41]:
question = """ I have 2 dogs, a Border Collie and a Scottish Terrier.
What is their combined weight"""
abot(question)

'Thought: To find the combined weight of the two dogs, I need to get the average weight for each breed using the `average_dog_weight` action, then sum those weights.\n\nAction: average_dog_weight: BorderCollie\nPAUSE'

In [42]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [43]:
abot(next_prompt)

'Action: average_dog_weight: ScottishTerrier\nPAUSE'

In [44]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [45]:
abot(next_prompt)

'Thought: Now that I have the average weights for both breeds, I can calculate their combined weight by adding the two values together.\n\nAction: calculate: 37 + 20\nPAUSE'

In [46]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [47]:
abot(next_prompt)

'Answer: The combined weight of your border collie and scottish terrier is 57 lbs.'

# Automate (Add loop)
It was manual agent, let's automate it and add loop

In [69]:
abot = Agent(prompt)
action_re = re.compile('^Action: (\\w+): (.*)$')
#it parse llm response and determine whether to take action or it is final answer

In [66]:
def query(question, max_turns=5):
    i=0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        result = bot(next_prompt)
        print(result)
        actions = [
        action_re.match(a)
        for a in result.split('\n')
        if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return
        

In [68]:
question = """I have 2 dogs, a Border Collie and a Scottish Terrier. \
What is their combined weight"""
query(question)

Thought: To find the combined weight of the two dogs, I need to get the average weight for each breed using the `average_dog_weight` action and then sum them up.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the average weight of a Border Collie, I need to get the average weight of a Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: With both average weights known, I can now calculate the combined weight by adding the two values together.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of your Border Collie and Scottish Terrier is 57 lbs.
